# Bird Name Generation using RNNs

In [ ]:
import numpy as np
import pandas as pd
import random
from collections import Counter
import matplotlib.pyplot as plt

## Reading the data

In [ ]:
# Reading the CSV file into a DatFrame
names_df = pd.read_csv('/kaggle/input/common-bird-names/birds.csv')
names_df

## Creating the input and labels

We'll append a `$` character at the end of each word to indicate the end of the sequence. `$` is chosen because it does not occur anywherwe in our data.

In [ ]:
names = names_df['Common Bird Names'].to_list()

# Appending a '$' character at end of each word to denote the end
names = [name.lower() + '$' for name in names]

print(f"Corpus length: {len(names)}")
print(f'Name at 0th position is: {names[0]}')

In [ ]:
# Preparting a set of all the characters
characters = set()

for name in names:
    for i, character in enumerate(name):
        characters.add(character)

characters = sorted(characters)

print(f"Number of characters: {len(characters)}")

In [ ]:
# Creating forward and reverse lookup tables from characters to indices
char_to_index = {c:i for i, c in enumerate(characters)}
index_to_char = {i:c for i, c in enumerate(characters)}

print(f'Index for the $ character is: {char_to_index["$"]}')
print(f'Character for the index {char_to_index["$"]} is: {index_to_char[char_to_index["$"]]}')

Since we are using Character Level RNNs, we will take current character as the input and the next character as its label that our model has to predict.

In [ ]:
max_len = 40
m = len(names)
char_dim = len(characters)

x = np.zeros((m, max_len, char_dim), dtype=np.bool)
y = np.zeros((m, max_len, char_dim), dtype=np.bool)

for i in range(m):
    name = names[i]
    for j in range(len(name)):
        x[i, j, char_to_index[name[j]]] = 1
        if j < len(name) - 1:
            y[i, j, char_to_index[name[j+1]]] = 1
            
print(f"Shape of x: {x.shape}")
print(f"Shape of y: {y.shape}")      

In [ ]:
from keras import Input
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Dropout
from keras.callbacks import LambdaCallback

## Model Creation

In [ ]:
# A simple model with one three layers of LSTM followed by an output Layer
model = Sequential([
    Input(shape=(max_len, char_dim)),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    Dense(char_dim, activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy', optimizer = 'adam')

In [ ]:
def make_samples(model):

    # Sampling over different temperatures. A lower temperature would give a more conservative output and a higher temperature would give more liberal ones.
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        x_pred = np.zeros((1, max_len, len(characters)))
        i = 0
        generated = ""
        end = False

        while not end:
            preds = model.predict(x_pred, verbose=0)[0, i]
            next_index = np.random.choice(range(len(characters)), p=preds)
            # Set next character as end of sequence caracter if max length is reached
            if i == max_len - 2:
                next_char = '$'
                end = True
            else:
                next_char = index_to_char[next_index]
            generated += next_char
            x_pred[0, i+1, next_index] = 1.0
            i += 1
            if next_char == '$':
                end = True
            
        print(f"Generated: {generated}")
        print()

In [ ]:
def generate_name_loop(epoch, _):
    if epoch % 10 == 0:
        
        print("="*50)
        print(f'Names generated after epoch {epoch}:')
        
        print()
        make_samples(model)
      
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)

In [ ]:
# Model training
epochs = 201
batch_size = 128

history = model.fit(x, y, batch_size=batch_size, epochs=epochs, callbacks=[name_generator], verbose=0)

We can see that the model is able to generate names close to real anmes with a lower temperature and generates some wild names with a higher temperature.

In [ ]:
loss = history.history["loss"]

plt.plot(range(len(loss)), loss)

## References:

1. https://keras.io/examples/generative/lstm_character_level_text_generation/
1. https://towardsdatascience.com/generating-pok%C3%A9mon-names-using-rnns-f41003143333